In [167]:
import urllib.request as urllib
import zipfile
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import matplotlib.pyplot as plt
import time
import numpy as np

In [4]:
# csv parameter options:
# csv=0 --> AMI State, Counties, Cities 2018.csv
# csv=1 --> AMI Census Tracts 2018.csv
# csv=2 --> FPL State, Counties, Cities 2018.csv
# csv=3 --> FPL Census Tracts 2018.csv
# csv=4 --> SMI State, Counties, Cities 2018.csv
# csv=5 --> SMI Census Tracts 2018.csv

def get_LEAD_DataFrame(state_abbrev, csv=3):
    url = 'https://data.openei.org/files/573/' + state_abbrev + '-2018-LEAD-data.zip'
    filehandle, _ = urllib.urlretrieve(url)
    zip_file_object = zipfile.ZipFile(filehandle, 'r')
    my_file = zip_file_object.namelist()[csv]
    df = pd.read_csv(zip_file_object.open(my_file))
    zip_file_object.close()
    return df

In [269]:
def process_numpy_all_categories(df_np,sa):
    columns = ['State','Tenure','Year Built','Structure Type','Heating Fuel',
           'Federal Poverty Line','Units','HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS']
    final_arr = [np.array([0 for _ in range(11)])]
    for t in tenures:
        for y in yearbuilts:
            for b in bldtypes:
                for h in heatfuels:
                    for f in fpls:
                        temp_df = df_np[(df_np[:,2]==t)&(df_np[:,3]==y)&(df_np[:,4]==b)&(df_np[:,5]==h)&(df_np[:,6]==f), :]
                        temp_arr = [np.array([sa, t, y, b, h, f]+[round(sum(temp_df[:,i]),3) for i in range(7,12)])]
                        final_arr = np.append(final_arr, temp_arr, axis=0)
    df_final = pd.DataFrame(final_arr, columns = columns)
    df_final.drop([0],inplace=True)
    hincp, elep, gasp, fulp = [],[],[],[]
    df_final['Household Income'] = [round(float(df_final['HINCP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
    df_final['Electricity Payments ($)'] = [round(float(df_final['ELEP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
    df_final['Gas Payments ($)'] = [round(float(df_final['GASP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
    df_final['Fuel Oil Payments ($)'] = [round(float(df_final['FULP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
    df_final = df_final.drop(columns=['HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS'])
    df_final['Energy Burden (% Income)'] = [round(sum(df_final.iloc[i,7:])/df_final.iloc[i,6],4)*100 for i in range(len(df_final.index))]
    
    return df_final

In [270]:
def process_numpy_single_category(df_np,sa):
    columns2 = ['State','Metric Type','Metric Name','Units','HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS']
    final_arr2 = [np.array([0 for _ in range(8)])]
    for t in tenures:
        temp_arr = [np.array([sa, 'Tenure', t]+[round(sum(df_np[(df_np[:,2]==t),:][:,i]),3) for i in range(7,12)])]
        final_arr2 = np.append(final_arr2, temp_arr, axis=0)
    for y in yearbuilts:
        temp_arr = [np.array([sa, 'Year Built', y]+[round(sum(df_np[(df_np[:,3]==y),:][:,i]),3) for i in range(7,12)])]
        final_arr2 = np.append(final_arr2, temp_arr, axis=0)
    for b in bldtypes:
        temp_arr = [np.array([sa, 'Building Type', b]+[round(sum(df_np[(df_np[:,4]==b),:][:,i]),3) for i in range(7,12)])]
        final_arr2 = np.append(final_arr2, temp_arr, axis=0)
    for h in heatfuels:
        temp_arr = [np.array([sa, 'Heating Fuel', h]+[round(sum(df_np[(df_np[:,5]==h),:][:,i]),3) for i in range(7,12)])]
        final_arr2 = np.append(final_arr2, temp_arr, axis=0)
    for f in fpls:
        temp_arr = [np.array([sa, 'Percent of FPL', f]+[round(sum(df_np[(df_np[:,6]==f),:][:,i]),3) for i in range(7,12)])]
        final_arr2 = np.append(final_arr2, temp_arr, axis=0)
    df_final2 = pd.DataFrame(final_arr2, columns = columns2)
    df_final2.drop([0],inplace=True)
    hincp, elep, gasp, fulp = [],[],[],[]
    df_final2['Household Income'] = [round(float(df_final2['HINCP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
    df_final2['Electricity Payments ($)'] = [round(float(df_final2['ELEP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
    df_final2['Gas Payments ($)'] = [round(float(df_final2['GASP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
    df_final2['Fuel Oil Payments ($)'] = [round(float(df_final2['FULP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
    df_final2 = df_final2.drop(columns=['HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS'])
    df_final2['Energy Burden (% Income)'] = [round(sum(df_final2.iloc[i,5:])/df_final2.iloc[i,4],4)*100 for i in range(len(df_final2.index))]
    
    return df_final2

In [271]:
def create_LEAD_state_csv(sa):
    
    st1 = time.time()
    df = get_LEAD_DataFrame('WY', csv=3)
    print('Time for CSV Download:', round(time.time()-st1,2),'Seconds')
    
    tenures, yearbuilts, bldtypes = df.TEN.unique(), df.YBL6.unique(), df.BLD.unique()
    heatfuels, fpls = df.HFL.unique(), df.FPL15.unique()
    
    df_np = df.to_numpy()
    
    st2 = time.time()
    df_all = process_numpy_all_categories(df_np, sa)
    print('Time for all Categories DF:', round(time.time()-st2,2),'Seconds')
    
    st3 = time.time()
    df_one = process_numpy_single_category(df_np, sa)
    print('Time for single category DF:', round(time.time()-st3,2),'Seconds')
    
    return df_all, df_one

In [275]:
df.FPL15.unique()

array(['0-100%', '100-150%', '150-200%', '200-400%', '400%+'],
      dtype=object)

In [228]:
tenures = df.TEN.unique()
yearbuilts = df.YBL6.unique()
bldtypes = df.BLD.unique()
heatfuels = df.HFL.unique()
fpls = df.FPL15.unique()

In [229]:
df_np = df.to_numpy()

In [241]:
columns = ['State','Tenure','Year Built','Structure Type','Heating Fuel',
           'Federal Poverty Line','Units','HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS']
final_arr = [np.array([0 for _ in range(11)])]
for t in tenures:
    for y in yearbuilts:
        for b in bldtypes:
            for h in heatfuels:
                for f in fpls:
                    temp_df = df_np[(df_np[:,2]==t)&(df_np[:,3]==y)&(df_np[:,4]==b)&(df_np[:,5]==h)&(df_np[:,6]==f), :]
                    temp_arr = [np.array([sa, t, y, b, h, f]+[round(sum(temp_df[:,i]),3) for i in range(7,12)])]
                    final_arr = np.append(final_arr, temp_arr, axis=0)
df_final = pd.DataFrame(final_arr, columns = columns)
df_final.drop([0],inplace=True)
hincp, elep, gasp, fulp = [],[],[],[]
df_final['Household Income'] = [round(float(df_final['HINCP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final['Electricity Payments ($)'] = [round(float(df_final['ELEP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final['Gas Payments ($)'] = [round(float(df_final['GASP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final['Fuel Oil Payments ($)'] = [round(float(df_final['FULP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final = df_final.drop(columns=['HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS'])
df_final['Energy Burden (% Income)'] = [round(sum(df_final.iloc[i,7:])/df_final.iloc[i,6],4)*100 for i in range(len(df_final.index))]

In [250]:
columns2 = ['State','Metric Type','Metric Name','Units','HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS']
final_arr2 = [np.array([0 for _ in range(8)])]
for t in tenures:
    temp_arr = [np.array([sa, 'Tenure', t]+[round(sum(df_np[(df_np[:,2]==t),:][:,i]),3) for i in range(7,12)])]
    final_arr2 = np.append(final_arr2, temp_arr, axis=0)
for y in yearbuilts:
    temp_arr = [np.array([sa, 'Year Built', y]+[round(sum(df_np[(df_np[:,3]==y),:][:,i]),3) for i in range(7,12)])]
    final_arr2 = np.append(final_arr2, temp_arr, axis=0)
for b in bldtypes:
    temp_arr = [np.array([sa, 'Building Type', b]+[round(sum(df_np[(df_np[:,4]==b),:][:,i]),3) for i in range(7,12)])]
    final_arr2 = np.append(final_arr2, temp_arr, axis=0)
for h in heatfuels:
    temp_arr = [np.array([sa, 'Heating Fuel', h]+[round(sum(df_np[(df_np[:,5]==h),:][:,i]),3) for i in range(7,12)])]
    final_arr2 = np.append(final_arr2, temp_arr, axis=0)
for f in fpls:
    temp_arr = [np.array([sa, 'Percent of FPL', f]+[round(sum(df_np[(df_np[:,6]==f),:][:,i]),3) for i in range(7,12)])]
    final_arr2 = np.append(final_arr2, temp_arr, axis=0)
df_final2 = pd.DataFrame(final_arr2, columns = columns2)
df_final2.drop([0],inplace=True)
hincp, elep, gasp, fulp = [],[],[],[]
df_final2['Household Income'] = [round(float(df_final2['HINCP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
df_final2['Electricity Payments ($)'] = [round(float(df_final2['ELEP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
df_final2['Gas Payments ($)'] = [round(float(df_final2['GASP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
df_final2['Fuel Oil Payments ($)'] = [round(float(df_final2['FULP*UNITS'][i])/float(df_final2['Units'][i]),2) if float(df_final2['Units'][i])>0 else 0 for i in df_final2.index]
df_final2 = df_final2.drop(columns=['HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS'])
df_final2['Energy Burden (% Income)'] = [round(sum(df_final2.iloc[i,5:])/df_final2.iloc[i,4],4)*100 for i in range(len(df_final2.index))]

In [262]:
df_final2['Energy Burden (% Income)'] = [round(sum(df_final2.iloc[i,5:])/df_final2.iloc[i,4],4)*100 for i in range(len(df_final2.index))]

In [263]:
df_final2.head(50)

,State,Metric Type,Metric Name,Units,Household Income,Electricity Payments ($),Gas Payments ($),Fuel Oil Payments ($),Energy Burden (% Income)
1,WY,Tenure,OWNER,160121.0,90467.06,1367.44,936.04,121.55,2.68
2,WY,Tenure,RENTER,70509.0,45710.51,842.34,465.96,36.34,2.94
3,WY,Year Built,1940-59,31910.673,62352.85,960.79,733.58,79.14,2.84
4,WY,Year Built,1960-79,64393.516,70599.55,1188.29,737.46,81.78,2.84
5,WY,Year Built,1980-99,60032.129,85197.89,1382.07,780.23,120.20,2.68
6,WY,Year Built,2000-09,36528.861,92961.51,1337.07,980.61,102.07,2.60
7,WY,Year Built,2010+,12590.63,93910.03,1091.21,613.62,61.97,1.88
8,WY,Year Built,BEFORE 1940,25174.192,58791.47,1017.77,852.14,99.66,3.35
9,WY,Building Type,1 ATTACHED,10842.996,73881.61,1036.82,409.88,12.55,1.98
10,WY,Building Type,1 DETACHED,159689.824,89491.19,1375.91,917.56,117.13,2.69


In [242]:
df_final = pd.DataFrame(final_arr, columns = columns)
df_final.drop([0],inplace=True)

In [246]:
hincp, elep, gasp, fulp = [],[],[],[]
df_final['Household Income'] = [round(float(df_final['HINCP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final['Electricity Payments ($)'] = [round(float(df_final['ELEP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final['Gas Payments ($)'] = [round(float(df_final['GASP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]
df_final['Fuel Oil Payments ($)'] = [round(float(df_final['FULP*UNITS'][i])/float(df_final['Units'][i]),2) if float(df_final['Units'][i])>0 else 0 for i in df_final.index]

In [247]:
df_weird = df_final[(df_final['Household Income']<3)&(df_final['Household Income']>0)]

In [248]:
df_weird.head()

,State,Tenure,Year Built,Structure Type,Heating Fuel,Federal Poverty Line,Units,HINCP*UNITS,ELEP*UNITS,GASP*UNITS,FULP*UNITS,Household Income,Electricity Payments ($),Gas Payments ($),Fuel Oil Payments ($)
3376,WY,RENTER,1960-79,3-4 UNIT,BOTTLED GAS,0-100%,1.144,2.018,1.699,0.0,0.0,1.76,1.49,0.0,0.0


In [234]:
df_final = df_final.drop(columns=['HINCP*UNITS','ELEP*UNITS','GASP*UNITS','FULP*UNITS'])

5400

In [233]:
df_final.head(50)

,State,Tenure,Year Built,Structure Type,Heating Fuel,Federal Poverty Line,Units,HINCP*UNITS,ELEP*UNITS,GASP*UNITS,FULP*UNITS,Household Income,Electricity Payments ($),Gas Payments ($),Fuel Oil Payments ($)
1,WY,OWNER,1940-59,1 ATTACHED,BOTTLED GAS,0-100%,0,0,0,0,0,0,0,0,0
2,WY,OWNER,1940-59,1 ATTACHED,BOTTLED GAS,100-150%,0,0,0,0,0,0,0,0,0
3,WY,OWNER,1940-59,1 ATTACHED,BOTTLED GAS,150-200%,0,0,0,0,0,0,0,0,0
4,WY,OWNER,1940-59,1 ATTACHED,BOTTLED GAS,200-400%,0,0,0,0,0,0,0,0,0
5,WY,OWNER,1940-59,1 ATTACHED,BOTTLED GAS,400%+,0,0,0,0,0,0,0,0,0
6,WY,OWNER,1940-59,1 ATTACHED,ELECTRICITY,0-100%,0,0,0,0,0,0,0,0,0
7,WY,OWNER,1940-59,1 ATTACHED,ELECTRICITY,100-150%,0,0,0,0,0,0,0,0,0
8,WY,OWNER,1940-59,1 ATTACHED,ELECTRICITY,150-200%,12,419937,4221,0,0,34994,351,0,0
9,WY,OWNER,1940-59,1 ATTACHED,ELECTRICITY,200-400%,3,110857,1114,0,0,36952,371,0,0
10,WY,OWNER,1940-59,1 ATTACHED,ELECTRICITY,400%+,0,0,0,0,0,0,0,0,0


In [84]:
df2.head(50)

,ABV,FIP,TEN,YBL6,BLD,HFL,FPL15,UNITS,HINCP*UNITS,ELEP*UNITS,GASP*UNITS,FULP*UNITS,HINCP UNITS,ELEP UNITS,GASP UNITS,FULP UNITS,HCOUNT,ECOUNT,GCOUNT,FCOUNT,HINCP,ELEP,GASP,FULP
33,WY,56001962700,OWNER,1940-59,1 DETACHED,BOTTLED GAS,200-400%,0.754773,5.879326e+04,1303.714885,4.364138e+03,1.380001e+02,0.754773,0.754773,0.754773,0.754773,0.920800,0.920800,0.920800,0.920800,77895.290969,1727.293947,5.782054e+03,1.828365e+02
34,WY,56001962700,OWNER,1940-59,1 DETACHED,BOTTLED GAS,400%+,0.976768,1.497897e+05,3293.159422,9.080882e+02,5.087993e+02,0.976768,0.976768,0.976768,0.976768,1.183200,0.683200,1.183200,0.683200,153352.468537,3371.487419,9.296871e+02,5.209011e+02
37,WY,56001962700,OWNER,1940-59,1 DETACHED,ELECTRICITY,150-200%,1.066896,2.833824e+04,2224.455130,1.714522e-08,1.351704e-09,1.066896,1.066896,1.066896,1.066896,1.371300,1.003000,1.371300,1.003000,26561.398524,2084.978910,1.607019e-08,1.266951e-09
38,WY,56001962700,OWNER,1940-59,1 DETACHED,ELECTRICITY,200-400%,2.701259,1.370554e+05,4652.745060,1.400466e+03,3.422362e-09,2.701259,2.701259,2.701259,2.701259,6.472000,6.148800,6.472000,5.648800,50737.601808,1722.435768,5.184493e+02,1.266951e-09
39,WY,56001962700,OWNER,1940-59,1 DETACHED,ELECTRICITY,400%+,1.508963,1.455199e+05,1730.638633,2.202678e+03,1.911782e-09,1.508963,1.508963,1.508963,1.508963,5.527200,3.527200,5.527200,3.527200,96437.007436,1146.905700,1.459729e+03,1.266951e-09
43,WY,56001962700,OWNER,1940-59,1 DETACHED,OTHER,200-400%,1.469229,6.087533e+04,180.477660,1.693045e+03,1.861441e-09,1.469229,1.469229,1.469229,1.469229,0.452000,0.452000,0.452000,0.452000,41433.520000,122.838338,1.152335e+03,1.266951e-09
50,WY,56001962700,OWNER,1940-59,1 DETACHED,UTILITY GAS,0-100%,2.876984,3.975972e+04,4382.181782,5.034106e+03,1.976442e+01,2.876984,2.876984,2.876984,2.876984,6.685000,6.185000,6.685000,6.185000,13819.931874,1523.186223,1.749786e+03,6.869839e+00
51,WY,56001962700,OWNER,1940-59,1 DETACHED,UTILITY GAS,100-150%,3.826936,9.388850e+04,5789.399132,2.975631e+03,3.605325e+01,3.826936,3.826936,3.826936,3.826936,6.927333,5.977133,6.927333,5.977133,24533.592900,1512.802556,7.775491e+02,9.420918e+00
52,WY,56001962700,OWNER,1940-59,1 DETACHED,UTILITY GAS,150-200%,3.043841,8.562516e+04,5407.184353,1.377527e+04,2.434933e+01,3.043841,3.043841,3.043841,3.043841,7.053267,5.816533,7.053267,5.816533,28130.632221,1776.434707,4.525623e+03,7.999542e+00
53,WY,56001962700,OWNER,1940-59,1 DETACHED,UTILITY GAS,200-400%,18.227901,9.775388e+05,22532.582351,2.353681e+04,1.202730e+03,18.227901,18.227901,18.227901,18.227901,36.820000,22.696533,36.820000,22.696533,53628.707538,1236.158890,1.291252e+03,6.598292e+01
